In [1]:
import tensorflow as tf

In [1]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Paths to the train, test, and validation data folders
train_dir = 'split_data/train'
test_dir = 'split_data/test'
val_dir = 'split_data/val'

In [3]:
# Image dimensions
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

# Batch size
batch_size = 32

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

Found 5536 images belonging to 15 classes.
Found 706 images belonging to 15 classes.
Found 686 images belonging to 15 classes.


In [6]:
# Model architecture
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
#pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator,
                    verbose=1)

Epoch 1/10
173/173 [==============================] - 365s 2s/step - loss: 1.8153 - accuracy: 0.3793 - val_loss: 1.7089 - val_accuracy: 0.4111
Epoch 2/10
173/173 [==============================] - 397s 2s/step - loss: 1.4864 - accuracy: 0.4809 - val_loss: 1.6792 - val_accuracy: 0.4140
Epoch 3/10
173/173 [==============================] - 366s 2s/step - loss: 1.3241 - accuracy: 0.5423 - val_loss: 1.6531 - val_accuracy: 0.4315
Epoch 4/10
173/173 [==============================] - 365s 2s/step - loss: 1.2173 - accuracy: 0.5836 - val_loss: 1.6013 - val_accuracy: 0.4548
Epoch 5/10
173/173 [==============================] - 397s 2s/step - loss: 1.1090 - accuracy: 0.6212 - val_loss: 1.5019 - val_accuracy: 0.4927
Epoch 6/10
173/173 [==============================] - 389s 2s/step - loss: 1.0423 - accuracy: 0.6404 - val_loss: 1.5718 - val_accuracy: 0.5000
Epoch 7/10
173/173 [==============================] - 368s 2s/step - loss: 0.9465 - accuracy: 0.6837 - val_loss: 1.5462 - val_accuracy: 0.5015

In [10]:
# Evaluate the model on the test data
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.4115718603134155
Test accuracy: 0.5368272066116333


In [17]:
from keras.utils import to_categorical

y_pred = model.predict(test_generator)

# Compute LWLRAP score
y_true = to_categorical(test_generator.labels, num_classes=15)
lwlrAP = label_ranking_average_precision_score(y_true, y_pred)
print('LWLRAP score:', lwlrAP)

# Compute top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(test_generator.labels, y_pred, k=5)
print('Top-5 categorical accuracy:', top5_accuracy)

23/23 [==============================] - 73s 3s/step
LWLRAP score: 0.3126224027782097
Top-5 categorical accuracy: 0.5424929178470255
